# Релиз 2.0 Обработка данных

In [415]:
import pandas as pd
import numpy as np 
import re

Считываем CSV файл и отображаем все колонки DataFrame.

In [416]:
df=pd.read_csv('~/real_estate/data/_data.csv', index_col=0)
pd.set_option('display.max_columns', None)
pd.reset_option('display.max_colwidth')
pd.set_option('display.float_format', '{:.2f}'.format)


Удаляем все адреса не относящиеся к москве.

In [417]:
df0=df[df['Адрес'].str.contains('Москва', case=False)].copy()

Разделение столбца "Цена" на категории.

In [418]:
df1 = df0.copy()
df1['Цена'] = df1['Цена'].transform(lambda x: x.split(','))
df1['price'] = df1['Цена'].transform(lambda x: next((i for i in x if 'месяц' in i), np.nan))
df1['utility_payments'] = df1['Цена'].transform(lambda x: next((i for i in x if 'Сумма' in i), np.nan))
df1['deposit'] = df1['Цена'].transform(lambda x: next((i for i in x if 'Залог' in i), np.nan))
df1['public_utilities'] = df1['Цена'].transform(lambda x: next((i for i in x if 'Коммунальные' in i), np.nan))
df1['lease_term'] = df1['Цена'].transform(lambda x: next((i for i in x if 'Срок' in i), np.nan))
df1['prepayment'] = df1['Цена'].transform(lambda x: next((i for i in x if 'Предоплата' in i), np.nan))


Конвертируем валюты и преобразуем в числовой тип

In [ ]:
def convert_currency(price_str):

    price_str = str(price_str).lower()
    if '$' in price_str in price_str:
        return 82
    elif '€' in price_str in price_str:
        return 93
    return 1


df1['price'] = (
    df1['price'].str.extract(r'([\d\s,]+\.?\d*)')[0] 
    .apply(pd.to_numeric, errors='coerce')
    * df1['price'].apply(convert_currency)  
)

<bound method NDFrame.describe of 0        500000.00
1        500000.00
2        500000.00
3        400000.00
4        225000.00
5        470000.00
6        350000.00
7        250000.00
8        130000.00
9        210000.00
10       290000.00
11       100000.00
12       200000.00
13       500000.00
14       450000.00
15       450000.00
16       120000.00
17       450000.00
18       180000.00
19       190000.00
20       205000.00
21       230000.00
22       590000.00
23       850000.00
24       190000.00
25       200000.00
26       200000.00
27       300000.00
28       400000.00
29       750000.00
30       870000.00
31       350000.00
32       400000.00
33       400000.00
34       590000.00
35       135000.00
36       150000.00
37       320000.00
38       400000.00
39       400000.00
40       450000.00
41       470000.00
42       590000.00
43       600000.00
44       850000.00
45       150000.00
46       195000.00
47       300000.00
48        90000.00
49       150000.00
50       250000.

Разделение столбца "Метро" на категории.

In [420]:
extracted = df1['Метро'].str.extract(r'(\d+)\sмин\s(пешком|на машине)')     # изменила время метро, учитывает, на машине 
extracted.columns = ['minutes', 'transport']
df1['time_metro'] = extracted.apply(
    lambda row: int(row['minutes']) * 3 if row['transport'] == 'на машине' 
    else int(row['minutes']) if pd.notna(row['minutes']) 
    else np.nan,
    axis=1)
df1['metro'] = df1['Метро'].transform(lambda x: x.split('(')[0].strip() if pd.notna(x) else np.nan)


Форматирование столбца "Площадь, м2" и приведение к общей площади, так как остальные данные невозможно идентифицировать.

In [421]:
df1['total_area'] = pd.to_numeric(df1['Площадь, м2'].str.extract(r'(\d+\.\d+)')[0])

Разделение столбца "Дом" на категории

In [422]:
df1['floor'] = df1['Дом'].transform(lambda x: x.split(',')[0].split('/')[0])
df1['all_floor'] = df1['Дом'].transform(lambda x: x.split(',')[0].split('/')[1])
df1['house'] = df1['Дом'].transform(       # изменила тут при преобразоании вначале был пробел
    lambda x: x.split(',')[1].strip() if len(x.split(',')) > 1 else np.nan
)
df1['floor'] = pd.to_numeric(df1['floor'], errors='coerce')
df1['all_floor'] = pd.to_numeric(df1['all_floor'], errors='coerce')

Приведение колонки "Количество комнат" к числовому значению и удаление неинформативного материала.

In [423]:
df1['rooms'] = pd.to_numeric(df1['Количество комнат'].str.extract(r'(\d+)')[0], errors='coerce')

Приводим колонку время до метро к числовому значению

In [424]:
df1['time_metro'] = pd.to_numeric(df1['time_metro'], errors='coerce')

Удаление всех столбцов повторяющих данные, не несущих информативносли или же не идентифицириумых.

In [425]:
df1.drop(columns= ['Название ЖК', 'Дом', 'Дополнительно', 'prepayment', 'utility_payments', 'lease_term', 'deposit', 'public_utilities', 'Количество комнат', 'Площадь, м2', 'Описание','Площадь комнат, м2', 'Мусоропровод', 'Метро', 'Высота потолков, м', 'Телефоны','Серия дома', 'Ссылка на объявление', 'Тип', 'Адрес', 'Цена'], inplace=True)
df1.head()

,ID объявления,Парковка,Ремонт,Балкон,Окна,Санузел,Можно с детьми/животными,Лифт,price,time_metro,metro,total_area,floor,all_floor,house,rooms
0,271271157,подземная,Дизайнерский,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Пасс (4), Груз (1)",500000.00,9.00,м. Смоленская,200.00,5,16,Монолитный,4.00
1,271634126,подземная,Дизайнерский,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Пасс (1), Груз (1)",500000.00,8.00,м. Смоленская,198.00,5,16,Монолитно-кирпичный,4.00
2,271173086,подземная,Евроремонт,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,Пасс (1),500000.00,7.00,м. Смоленская,200.00,5,16,NaN,4.00
3,272197456,подземная,Евроремонт,NaN,На улицу и двор,Совмещенный (3),Можно с животными,Пасс (1),400000.00,3.00,м. Смоленская,170.00,5,6,NaN,4.00
4,273614615,NaN,Евроремонт,NaN,На улицу и двор,Совмещенный (2),NaN,"Пасс (1), Груз (1)",225000.00,7.00,м. Арбатская,58.00,12,26,Панельный,2.00


Переименовываем столбцы по стандарту 


In [426]:
column_rename = {
    'ID  объявления': 'ID',
    'Парковка': 'parking',
    'Ремонт': 'renovation',
    'Балкон': 'balcony',
    'Окна': 'windows',
    'Санузел': 'bathroom',
    'Можно с детьми/животными': 'children_pets_allowed',
    'Лифт': 'elevator',
}
df1 = df1.rename(columns=column_rename)
df1.head()

,ID,parking,renovation,balcony,windows,bathroom,children_pets_allowed,elevator,price,time_metro,metro,total_area,floor,all_floor,house,rooms
0,271271157,подземная,Дизайнерский,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Пасс (4), Груз (1)",500000.00,9.00,м. Смоленская,200.00,5,16,Монолитный,4.00
1,271634126,подземная,Дизайнерский,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Пасс (1), Груз (1)",500000.00,8.00,м. Смоленская,198.00,5,16,Монолитно-кирпичный,4.00
2,271173086,подземная,Евроремонт,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,Пасс (1),500000.00,7.00,м. Смоленская,200.00,5,16,NaN,4.00
3,272197456,подземная,Евроремонт,NaN,На улицу и двор,Совмещенный (3),Можно с животными,Пасс (1),400000.00,3.00,м. Смоленская,170.00,5,6,NaN,4.00
4,273614615,NaN,Евроремонт,NaN,На улицу и двор,Совмещенный (2),NaN,"Пасс (1), Груз (1)",225000.00,7.00,м. Арбатская,58.00,12,26,Панельный,2.00


Работаем с NaN значениями

In [427]:
df1.isna().sum().to_frame()


,0
ID,0
parking,11174
renovation,2463
balcony,6630
windows,5150
bathroom,2041
children_pets_allowed,4915
elevator,4192
price,0
time_metro,884


NaN в количествах комнат заменяет на 0, так как это студия (проаналировали несколько ссылок в датасете)

In [428]:
df1['rooms'] = df1['rooms'].fillna(0)
df1['rooms'].value_counts()

rooms
2.00    7407
1.00    6655
3.00    3668
4.00    1011
0.00     535
5.00     333
6.00     128
Name: count, dtype: int64

NaN в балконах, парковках и лифтах заменяем на 'отсутствует', так как в данном контексте это и означает

In [429]:
df1['parking'] = df1['parking'].fillna('отсутсвует').astype('category')

In [430]:
df1['elevator'] = df1['elevator'].fillna('отсутсвует').astype('category')

In [431]:
df1['balcony'] = df1['balcony'].fillna('отсутсвует').astype('category')

- Упростили данные (объединили "Без ремонта (136) с "Косметическим")

- Заполняем пропуски самым популярным типом ремонта для аналогичных по цене квартир

In [432]:
df1['renovation'].value_counts(dropna=False, normalize=True)   # какие варианты ремонта есть в данных и как часто они встречаются (в процентах)

renovation
Косметический   0.37
Евроремонт      0.35
Дизайнерский    0.14
NaN             0.12
Без ремонта     0.01
Name: proportion, dtype: float64

In [433]:
df1['renovation'] = df1['renovation'].replace('Без ремонта', 'Косметический')

df1['renovation'] = (
    df1.groupby(pd.qcut(df1['price'], 5))['renovation']
    .transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))
)

df1['renovation'].value_counts(dropna=False, normalize=True)

/tmp/ipykernel_10047/2090160448.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df1.groupby(pd.qcut(df1['price'], 5))['renovation']


renovation
Косметический   0.42
Евроремонт      0.41
Дизайнерский    0.17
Name: proportion, dtype: float64

Заполняем NaN в столбце windows на основе средней цены квартиры. Идея: "Похожие по цене квартиры обычно имеют одинаковый тип окон"

In [434]:
df1['windows'].value_counts()

windows
Во двор            9708
На улицу и двор    2762
На улицу           2117
Name: count, dtype: int64

In [435]:
avg_prices = df1.groupby('windows')['price'].median().to_dict()

def fill_by_price(row):
    if pd.isna(row['windows']):
        closest_window = min(avg_prices.keys(), key=lambda x: abs(avg_prices[x] - row['price']))
        return closest_window
    return row['windows']

df1['windows'] = df1.apply(fill_by_price, axis=1)
df1['windows'].value_counts()

windows
Во двор            12657
На улицу и двор     4150
На улицу            2930
Name: count, dtype: int64

Заполняем NaN в столбце bathroom с учетом количеством комнат

In [436]:
def fill_group(group):

    mode_values = group.mode()
    if not mode_values.empty:
        return group.fillna(mode_values.iloc[0])
    return group


df1['bathroom'] = df1.groupby('rooms')['bathroom'].transform(fill_group)


if df1['bathroom'].isna().any():
    overall_mode = df1['bathroom'].mode()
    if not overall_mode.empty:
        df1['bathroom'] = df1['bathroom'].fillna(overall_mode.iloc[0])

Заполняем NaN в столбце house по этажам

In [437]:
df1['house'].value_counts()

house
Панельный              6679
Кирпичный              3696
Монолитный             3615
Блочный                1689
Монолитно-кирпичный     872
Сталинский              141
старый фонд              68
Деревянный                5
Щитовой                   1
Name: count, dtype: int64

In [438]:
FLOOR_RULES = {
    'Деревянный': (1, 1),        
    'Щитовой': (1, 1),           
    'старый фонд': (1, 4),       
    'Сталинский': (2, 9),        
    'Кирпичный': (2, 5),       
    'Блочный': (5, 9),          
    'Панельный': (5, 16),       
    'Монолитно-кирпичный': (10, 25),  
    'Монолитный': (17, 100)     
}

def fill_house_by_floors(df):

    df_filled = df.copy()
    
 
    for house_type, (min_floor, max_floor) in FLOOR_RULES.items():
        mask = (
            df_filled['house'].isna() & 
            df_filled['all_floor'].between(min_floor, max_floor)
        )
        

        if house_type in ('Деревянный', 'Щитовой'):
            available = min(mask.sum(), (df_filled['all_floor'] == 1).sum())
            if available > 0:
                fill_idx = df_filled[mask].head(available).index
                df_filled.loc[fill_idx, 'house'] = house_type
        else:
            df_filled.loc[mask, 'house'] = house_type
    

    if df_filled['house'].isna().any():
        house_dist = df_filled['house'].value_counts(normalize=True)
        fill_values = np.random.choice(
            house_dist.index,
            size=df_filled['house'].isna().sum(),
            p=house_dist.values
        )
        df_filled.loc[df_filled['house'].isna(), 'house'] = fill_values
    
    return df_filled


df1 = fill_house_by_floors(df1)



df1['house'].value_counts()

house
Панельный              7418
Монолитный             3882
Кирпичный              3696
Блочный                1689
Монолитно-кирпичный    1663
Сталинский             1189
старый фонд             193
Деревянный                6
Щитовой                   1
Name: count, dtype: int64

Заполняем NaN у children_pets_allowed (отсутсвие данных это есть требуемое значение)

In [439]:
df1['children_pets_allowed'] = df1['children_pets_allowed'].fillna('ни то и ни то')

Заполняем пропуски у метро

In [440]:
df1['metro'] = df1['metro'].replace('', np.nan)
df1['metro'] = df1.groupby('price')['metro'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)
)
df1.isna().sum().to_frame()

/tmp/ipykernel_10047/2992113798.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)


,0
ID,0
parking,0
renovation,0
balcony,0
windows,0
bathroom,0
children_pets_allowed,0
elevator,0
price,0
time_metro,884


Заполним пропуски у время до метро

In [441]:
metro_time_avg = df1.dropna(subset=['metro']).groupby('metro')['time_metro'].median().to_dict()


df1['time_metro'] = df1.apply(
    lambda row: (
        metro_time_avg.get(row['metro'], row['time_metro'])  
        if pd.notna(row['metro'])
        else row['time_metro'] 
    ),
    axis=1
)
df1['time_metro'].isna().sum()

np.int64(9)

In [413]:
mode_time = df1['time_metro'].mode()[0]

df1['time_metro'] = df1['time_metro'].fillna(mode_time)

mode_time = df1['metro'].mode()[0]

df1['metro'] = df1['metro'].fillna(mode_time) 

df1.isna().sum().to_frame()

,0
ID,0
parking,0
renovation,0
balcony,0
windows,0
bathroom,0
children_pets_allowed,0
elevator,0
price,0
time_metro,0
